In [ ]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import seaborn as sns
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from network import create_model, create_dataloaders

In [ ]:
# 检查PyTorch是否正确编译了CUDA
print(f"PyTorch是否编译了CUDA: {torch.cuda.is_available()}")
print(f"PyTorch CUDA版本: {torch.version.cuda}")

In [ ]:
# 1. 选择训练方式
sizes = [128, 256, 512, 1024, 2048, 4096] # 基于符号长度进行训练
snr_list = [-10, -5, 0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50] # 基于信噪比进行训练
items = snr_list

In [ ]:
# 加载数据
X_train, y_train = {}, {}
X_test, y_test = {}, {}

for item in items:
# for items in sizes:
    # 训练集
    df_train = shuffle(pd.read_csv(f'../preprocessed/combined_features_{item}.csv'), random_state=42)
    y_train = df_train['label']
    X_train = df_train.drop('label', axis=1).applymap(lambda x: complex(x)).values
    
    # 测试集
    df_test = shuffle(pd.read_csv(f'../preprocessed/combined_features_{item}_test.csv'), random_state=42)
    y_test = df_test['label']
    X_test = df_test.drop('label', axis=1).applymap(lambda x: complex(x)).values

In [ ]:
# 2. 创建数据加载器
batch_size = 32
train_loader, test_loader, (scaler_real, scaler_imag) = create_dataloaders(X_train, y_train, X_test, y_test, batch_size)

In [ ]:
# 3. 创建模型
num_features = X_train.shape[1]
num_classes = len(np.unique(y_train))
model = create_model(num_features, num_classes)

In [ ]:
# 4. 设置训练参数
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

# 确保模型在GPU上
model = create_model(num_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=3)

In [ ]:
# 5. 训练模型
num_epochs = 30
best_acc = 0.0

accuracies = []


for item in items:
    print(f"\n=== Training for items {item} ===")
    for epoch in range(1, num_epochs + 1):
        # 训练阶段
        model.train()
        total_loss = 0
        
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        # 验证阶段
        model.eval()
        correct = 0
        total = 0
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                outputs = model(x)
                _, predicted = torch.max(outputs.data, 1)
                total += y.size(0)
                correct += (predicted == y).sum().item()
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(y.cpu().numpy())
        
        acc = correct / total
        scheduler.step(acc)
        
        if acc > best_acc:
            best_acc = acc
            # 保存最佳模型
            # torch.save(model.state_dict(), f'./model_save/ComplexCNN_{item}.pth')
        
        print(f'Epoch {epoch}/{num_epochs}: Loss={total_loss/len(train_loader):.4f}, Acc={acc:.4f}, Best Acc={best_acc:.4f}')

    print(f"最终最佳准确率: {best_acc:.4f}")

In [ ]:
# 6. 绘制混淆矩阵（可选）
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('混淆矩阵', fontproperties="SimSun")
plt.ylabel('真实类别', fontproperties="SimSun")
plt.xlabel('预测类别', fontproperties="SimSun")
plt.xticks(range(4), ['bladerf', 'hackrf0', 'hackrf1', 'limesdr'])
plt.yticks(range(4), ['bladerf', 'hackrf0', 'hackrf1', 'limesdr'])
plt.savefig(f'./img/confusion_matrix_{item}.png', dpi=300, bbox_inches='tight')
plt.show()